In [232]:
import statsmodels
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [233]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [234]:
data = pd.read_csv('botswana.tsv', '\t')

In [235]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [236]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4361 entries, 0 to 4360
Data columns (total 15 columns):
ceb         4361 non-null int64
age         4361 non-null int64
educ        4361 non-null int64
religion    4361 non-null object
idlnchld    4241 non-null float64
knowmeth    4354 non-null float64
usemeth     4290 non-null float64
evermarr    4361 non-null int64
agefm       2079 non-null float64
heduc       1956 non-null float64
urban       4361 non-null int64
electric    4358 non-null float64
radio       4359 non-null float64
tv          4359 non-null float64
bicycle     4358 non-null float64
dtypes: float64(9), int64(5), object(1)
memory usage: 511.1+ KB


In [237]:
data.describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
count,4361.000000,4361.000000,4361.000000,4241.000000,4354.000000,4290.000000,4361.000000,2079.000000,1956.000000,4361.000000,4358.000000,4359.000000,4359.000000,4358.000000
mean,2.441642,27.405182,5.855996,4.615892,0.963252,0.577622,0.476726,20.686388,5.144683,0.516625,0.140202,0.701766,0.092911,0.275815
std,2.406861,8.685233,3.927075,2.219303,0.188164,0.493996,0.499515,5.002383,4.803028,0.499781,0.347236,0.457535,0.290341,0.446975
min,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,3.000000,3.000000,1.000000,0.000000,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,26.000000,7.000000,4.000000,1.000000,1.000000,0.000000,20.000000,6.000000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,4.000000,33.000000,8.000000,6.000000,1.000000,1.000000,1.000000,23.000000,8.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,13.000000,49.000000,20.000000,20.000000,1.000000,1.000000,1.000000,46.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [238]:
print(data.religion.value_counts(),'\n\n',
      'Length:', len(data.religion.value_counts()))

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64 

 Length: 4


In [239]:
print('Original shape:', data.shape)
print('After drop NaN:', data.dropna().shape)

Original shape: (4361, 15)
After drop NaN: (1834, 15)


In [240]:
data['nevermarr'] = data['agefm'].apply(lambda x: 1 if np.isnan(x) else 0)

In [241]:
data.drop(['evermarr'], axis=1, inplace=True)

In [242]:
c_age_fm = 0
data['agefm'] = data['agefm'].fillna(c_age_fm)

In [243]:
c_heduc_1 = -1
data.loc[(data['nevermarr'] == 1) & (data['heduc'].apply(np.isnan)), 'heduc'] = c_heduc_1

In [244]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [245]:
np.isnan(data['heduc']).sum()

123

In [246]:
c_idlnchld = -1
c_heduc_2 = -2
c_usemeth = -1

data['idlnchld_noans'] = data['idlnchld'].apply(lambda x: 1 if np.isnan(x) else 0)
data['heduc_noans'] = data['heduc'].apply(lambda x: 1 if np.isnan(x) else 0)
data['usemeth_noans'] = data['usemeth'].apply(lambda x: 1 if np.isnan(x) else 0)

data['idlnchld'] = data['idlnchld'].fillna(c_idlnchld)
data['heduc'] = data['heduc'].fillna(c_heduc_2)
data['usemeth'] = data['usemeth'].fillna(c_usemeth)

In [247]:
data = data.dropna()

In [248]:
data.shape[0] * data.shape[1]

78264

In [253]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth +'\
             'agefm + heduc + urban + electric + radio + tv + bicycle +'\
             'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m1.fit()
print (fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Mon, 18 Mar 2019   Prob (F-statistic):               0.00
Time:                        21:04:59   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [254]:
print ('Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `het_breushpagan` is deprecated, use `het_breuschpagan` instead!
Use het_breuschpagan, het_breushpagan will be removed in 0.9 
(Note: misspelling missing 'c')
  """Entry point for launching an IPython kernel.


In [255]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth +'\
             'agefm + heduc + urban + electric + bicycle +'\
             'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m2.fit(cov_type='HC1')
print (fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Mon, 18 Mar 2019   Prob (F-statistic):               0.00
Time:                        21:05:19   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [265]:
print ('Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `het_breushpagan` is deprecated, use `het_breuschpagan` instead!
Use het_breuschpagan, het_breushpagan will be removed in 0.9 
(Note: misspelling missing 'c')
  """Entry point for launching an IPython kernel.


In [256]:
print ("F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit()))

F=0.919236, p=0.467231, k1=5.000000


In [257]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth +'\
             'agefm + heduc + urban + electric + bicycle +'\
             'nevermarr + idlnchld_noans + heduc_noans', 
             data=data)
fitted = m3.fit(cov_type='HC1')
print (fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.4
Date:                Mon, 18 Mar 2019   Prob (F-statistic):               0.00
Time:                        21:07:15   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.262     -4.

In [266]:
print ('Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `het_breushpagan` is deprecated, use `het_breuschpagan` instead!
Use het_breuschpagan, het_breushpagan will be removed in 0.9 
(Note: misspelling missing 'c')
  """Entry point for launching an IPython kernel.


In [260]:
print ("F=%f, p=%f, k1=%f" % m2.fit().compare_f_test(m3.fit()))

F=92.890582, p=0.000000, k1=2.000000


In [264]:
m2.fit().compare_f_test(m3.fit())[1]

3.1552009480371243e-40

In [272]:
data[data['nevermarr'] == 1].describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
count,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.0,2278.0,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.0,2278.000000,2278.0,2278.000000
mean,1.287972,23.157155,6.636084,3.983758,0.964004,0.496927,0.0,-1.0,0.502195,0.118086,0.703687,0.071993,0.253292,1.0,0.017120,0.0,0.009219
std,1.636632,7.154783,3.467100,2.015138,0.186322,0.518214,0.0,0.0,0.500105,0.322781,0.456731,0.258533,0.434992,0.0,0.129748,0.0,0.095591
min,0.000000,15.000000,0.000000,-1.000000,0.000000,-1.000000,0.0,-1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.000000
25%,0.000000,18.000000,5.000000,3.000000,1.000000,0.000000,0.0,-1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.000000
50%,1.000000,21.000000,7.000000,4.000000,1.000000,1.000000,0.0,-1.0,1.000000,0.000000,1.000000,0.000000,0.000000,1.0,0.000000,0.0,0.000000
75%,2.000000,27.000000,9.000000,5.000000,1.000000,1.000000,0.0,-1.0,1.000000,0.000000,1.000000,0.000000,1.000000,1.0,0.000000,0.0,0.000000
max,11.000000,49.000000,19.000000,20.000000,1.000000,1.000000,0.0,-1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.0,1.000000


In [274]:
data[data['nevermarr'] == 0].describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
count,2070.000000,2070.000000,2070.00000,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000,2070.000000,2070.0,2070.000000,2070.000000,2070.000000
mean,3.702415,32.055556,5.01256,4.997101,0.962802,0.616425,20.691304,4.735266,0.533816,0.164251,0.699517,0.115942,0.299517,0.0,0.038164,0.058937,0.021739
std,2.473122,7.764931,4.20934,2.612013,0.189293,0.529206,5.008149,4.954843,0.498976,0.370593,0.458579,0.320233,0.458157,0.0,0.191639,0.235564,0.145866
min,0.000000,16.000000,0.00000,-1.000000,0.000000,-1.000000,10.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,2.000000,26.000000,0.00000,4.000000,1.000000,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
50%,3.000000,31.000000,5.00000,5.000000,1.000000,1.000000,20.000000,5.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
75%,5.000000,38.000000,7.00000,6.000000,1.000000,1.000000,23.000000,7.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000
max,13.000000,49.000000,20.00000,20.000000,1.000000,1.000000,46.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000
